## Title


### Objective




In [1]:
# standard import
import pandas as pd
import requests
import json
from flatten_json import flatten
import csv
from zipfile import ZipFile
from io import BytesIO, StringIO

import psycopg2

### 1. Retrieving data

### HERE
- add notes and details
- create scripts for API and data into postgres database
- create notebook for analysis and presentation

#### From World Bank Country API  
From the API documentation to retrieve the list of all the countries the url is 'http://api.worldbank.org/v2/country'; the data are retrieved in json format

In [2]:
WB_COUNTRY_URL = 'http://api.worldbank.org/v2/country?format=json'
CSV_GDP_URL = 'https://databank.worldbank.org/data/download/GDP.csv'
CSV_WDI_URL = 

In [2]:
try:
    r = requests.get(WB_COUNTRY_URL)
    r.raise_for_status()
except requests.exceptions.HTTPError as e:
    raise SystemExit(e)

Checking the result in json format

In [3]:
json_result = r.json()

In [4]:
len(json_result)

2

In [5]:
json_result[0]

{'page': 1, 'pages': 6, 'per_page': '50', 'total': 297}

In [6]:
json_result[1][0]

{'id': 'ABW',
 'iso2Code': 'AW',
 'name': 'Aruba',
 'region': {'id': 'LCN',
  'iso2code': 'ZJ',
  'value': 'Latin America & Caribbean '},
 'adminregion': {'id': '', 'iso2code': '', 'value': ''},
 'incomeLevel': {'id': 'HIC', 'iso2code': 'XD', 'value': 'High income'},
 'lendingType': {'id': 'LNX', 'iso2code': 'XX', 'value': 'Not classified'},
 'capitalCity': 'Oranjestad',
 'longitude': '-70.0167',
 'latitude': '12.5167'}

There are 6 pages of data to be retrieved and it is required to flatten the json results.

### HERE

In [7]:
# reduce two cells and update code

all_result = []

for page in range(1,7):
    url = f'http://api.worldbank.org/v2/country?format=json&page={page}'
    data = requests.get(url)
    res = data.json()
    all_result.append(json.loads(data.text)[1])

In [8]:
dfs = []

for df_res in range(0, len(all_result)):
    dic_flat = [flatten(d) for d in all_result[df_res]]
    df = pd.DataFrame(dic_flat)
    dfs.append(df)
    
df_country = pd.concat(dfs)

In [9]:
df_country.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 297 entries, 0 to 46
Data columns (total 18 columns):
id                      297 non-null object
iso2Code                297 non-null object
name                    297 non-null object
region_id               297 non-null object
region_iso2code         297 non-null object
region_value            297 non-null object
adminregion_id          297 non-null object
adminregion_iso2code    297 non-null object
adminregion_value       297 non-null object
incomeLevel_id          297 non-null object
incomeLevel_iso2code    297 non-null object
incomeLevel_value       297 non-null object
lendingType_id          297 non-null object
lendingType_iso2code    297 non-null object
lendingType_value       297 non-null object
capitalCity             297 non-null object
longitude               297 non-null object
latitude                297 non-null object
dtypes: object(18)
memory usage: 44.1+ KB


As expected we have a total of 297 rows. Let's look at the data to understand what needs to be stored in the database.

In [10]:
df_country.head(10)

,id,iso2Code,name,region_id,region_iso2code,region_value,adminregion_id,adminregion_iso2code,adminregion_value,incomeLevel_id,incomeLevel_iso2code,incomeLevel_value,lendingType_id,lendingType_iso2code,lendingType_value,capitalCity,longitude,latitude
0,ABW,AW,Aruba,LCN,ZJ,Latin America & Caribbean,,,,HIC,XD,High income,LNX,XX,Not classified,Oranjestad,-70.0167,12.5167
1,AFG,AF,Afghanistan,SAS,8S,South Asia,SAS,8S,South Asia,LIC,XM,Low income,IDX,XI,IDA,Kabul,69.1761,34.5228
2,AFR,A9,Africa,NA,NA,Aggregates,,,,NA,NA,Aggregates,,,Aggregates,,,
3,AGO,AO,Angola,SSF,ZG,Sub-Saharan Africa,SSA,ZF,Sub-Saharan Africa (excluding high income),LMC,XN,Lower middle income,IBD,XF,IBRD,Luanda,13.242,-8.81155
4,ALB,AL,Albania,ECS,Z7,Europe & Central Asia,ECA,7E,Europe & Central Asia (excluding high income),UMC,XT,Upper middle income,IBD,XF,IBRD,Tirane,19.8172,41.3317
5,AND,AD,Andorra,ECS,Z7,Europe & Central Asia,,,,HIC,XD,High income,LNX,XX,Not classified,Andorra la Vella,1.5218,42.5075
6,ARB,1A,Arab World,NA,NA,Aggregates,,,,NA,NA,Aggregates,,,Aggregates,,,
7,ARE,AE,United Arab Emirates,MEA,ZQ,Middle East & North Africa,,,,HIC,XD,High income,LNX,XX,Not classified,Abu Dhabi,54.3705,24.4764
8,ARG,AR,Argentina,LCN,ZJ,Latin America & Caribbean,LAC,XJ,Latin America & Caribbean (excluding high income),UMC,XT,Upper middle income,IBD,XF,IBRD,Buenos Aires,-58.4173,-34.6118
9,ARM,AM,Armenia,ECS,Z7,Europe & Central Asia,ECA,7E,Europe & Central Asia (excluding high income),UMC,XT,Upper middle income,IBD,XF,IBRD,Yerevan,44.509,40.1596


In [11]:
drop_col_country = ['adminregion_id', 'adminregion_iso2code','adminregion_value', 'lendingType_id', 'lendingType_iso2code', 'lendingType_value']

In [12]:
df_country.drop(drop_col_country, axis=1, inplace = True)

In [13]:
df_country.columns

Index(['id', 'iso2Code', 'name', 'region_id', 'region_iso2code',
       'region_value', 'incomeLevel_id', 'incomeLevel_iso2code',
       'incomeLevel_value', 'capitalCity', 'longitude', 'latitude'],
      dtype='object')

In [14]:
# check the resulting dataframe
df_country.head()

,id,iso2Code,name,region_id,region_iso2code,region_value,incomeLevel_id,incomeLevel_iso2code,incomeLevel_value,capitalCity,longitude,latitude
0,ABW,AW,Aruba,LCN,ZJ,Latin America & Caribbean,HIC,XD,High income,Oranjestad,-70.0167,12.5167
1,AFG,AF,Afghanistan,SAS,8S,South Asia,LIC,XM,Low income,Kabul,69.1761,34.5228
2,AFR,A9,Africa,NA,NA,Aggregates,NA,NA,Aggregates,,,
3,AGO,AO,Angola,SSF,ZG,Sub-Saharan Africa,LMC,XN,Lower middle income,Luanda,13.242,-8.81155
4,ALB,AL,Albania,ECS,Z7,Europe & Central Asia,UMC,XT,Upper middle income,Tirane,19.8172,41.3317


### From World Bank Data Catalog

Retrieve two files:
1. Global Economic Prospects dataset where we find the percentage of grouth per year for each country
2. GDP ranking: dataset where we find the ranking and the amount of GDP for each country for the year 2019

#### 1. GEP
The files to download are zipped, using zipfile library

In [15]:
CSV_GEP_URL = 'https://databank.worldbank.org/data/download/GEP_CSV.zip'

try:
    r2 = requests.get(CSV_GEP_URL)
    r2.raise_for_status()
except requests.exceptions.HTTPError as e:
    raise SystemExit(e)

In [16]:
zip_file = ZipFile(BytesIO(r2.content))

In [17]:
zip_file.namelist()

['GEPData.csv', 'GEPSeries.csv']

The zip dowload contains two csv files: the one we are interested is 'GEPData.csv'

In [18]:
df_csv_gep = pd.read_csv(zip_file.open(zip_file.namelist()[0]))

Check the data on from the csv file and their types

In [19]:
df_csv_gep.head(5)

,Country Name,Country Code,Indicator Name,Indicator Code,1999,2000,2001,2002,2003,2004,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 28
0,Advanced Economies,AME,"GDP growth, constant 2010 USD",NYGDPMKTPKDZ,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.2,1.6,-5.4,3.3,3.5,NaN
1,East Asia and Pacific,EAA,"GDP growth, constant 2010 USD",NYGDPMKTPKDZ,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6.3,5.8,0.9,7.4,5.2,NaN
2,Emerging Markets and Developing Economies (EMDE),EMD,"GDP growth, constant 2010 USD",NYGDPMKTPKDZ,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4.3,3.6,-2.6,5.0,4.2,NaN
3,Euro Area,E19,"GDP growth, constant 2010 USD",NYGDPMKTPKDZ,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.9,1.3,-7.4,3.6,4.0,NaN
4,Europe and Central Asia,ECH,"GDP growth, constant 2010 USD",NYGDPMKTPKDZ,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.4,2.3,-2.9,3.3,3.9,NaN


In [20]:
df_csv_gep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146 entries, 0 to 145
Data columns (total 29 columns):
Country Name      146 non-null object
Country Code      146 non-null object
Indicator Name    146 non-null object
Indicator Code    146 non-null object
1999              0 non-null float64
2000              0 non-null float64
2001              0 non-null float64
2002              0 non-null float64
2003              0 non-null float64
2004              0 non-null float64
2005              0 non-null float64
2006              0 non-null float64
2007              0 non-null float64
2008              0 non-null float64
2009              0 non-null float64
2010              0 non-null float64
2011              0 non-null float64
2012              0 non-null float64
2013              0 non-null float64
2014              0 non-null float64
2015              0 non-null float64
2016              0 non-null float64
2017              0 non-null float64
2018              146 non-null float64
2

The column that represent the values from the year 1999 to 2017 and the 'Unnamed: 28' column contain only NAs and will not be considered for further analysis.

In [21]:
df_csv_gep['Indicator Name'].value_counts()

GDP growth, constant 2010 USD    146
Name: Indicator Name, dtype: int64

In [22]:
df_csv_gep['Indicator Code'].value_counts()

NYGDPMKTPKDZ    146
Name: Indicator Code, dtype: int64

The column 'Indicator Name' and 'Indicator Code' contain the same value and do not contains usefull information, they will not be considered for further analysis.

In [23]:
drop_col_gep = list(df_csv_gep.loc[:, 'Indicator Name':'2017'].columns)
drop_col_gep.append('Unnamed: 28')

In [24]:
print(drop_col_gep)

['Indicator Name', 'Indicator Code', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', 'Unnamed: 28']


In [25]:
df_csv_gep.drop(drop_col_gep, axis=1, inplace = True)

In [26]:
#Renaming the columns for data modelling
df_csv_gep.columns = ['CountryName', 'CountryCode', 'Y_2018', 'Y_2019', 'Y_2020', 'Y_2021', 'Y_2022']

In [27]:
# check the resulting dataframe
df_csv_gep.head()

,CountryName,CountryCode,Y_2018,Y_2019,Y_2020,Y_2021,Y_2022
0,Advanced Economies,AME,2.2,1.6,-5.4,3.3,3.5
1,East Asia and Pacific,EAA,6.3,5.8,0.9,7.4,5.2
2,Emerging Markets and Developing Economies (EMDE),EMD,4.3,3.6,-2.6,5.0,4.2
3,Euro Area,E19,1.9,1.3,-7.4,3.6,4.0
4,Europe and Central Asia,ECH,3.4,2.3,-2.9,3.3,3.9


#### 2. GDP
The files to download are zipped, using zipfile library

In [28]:
CSV_GDP_URL = 'https://databank.worldbank.org/data/download/GDP.csv'

try:
    r3 = requests.get(CSV_GDP_URL)
    r3.raise_for_status()
except requests.exceptions.HTTPError as e:
    raise SystemExit(e)

In [29]:
file_gdp = r3.content.decode('utf8')

In [30]:
df_csv_gdp = pd.read_csv(StringIO(file_gdp))

In [31]:
df_csv_gdp.head()

,Unnamed: 0,Gross domestic product 2019,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,(millions of,NaN
2,NaN,Ranking,NaN,Economy,US dollars),NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,USA,1,NaN,United States,"21,433,226",NaN


The first few lines contain a title, that is read as header of column 1, blank lines and the name of the columns are spread in lines two and three: these need to be skipped and the columns need to be renamed. Columns Unnamed :2 and Unnamed :5 need to be dropped.
The file could simply be retrieved with the pandas method read_csv

In [32]:
col_names_gdp = ['country_code', 'ranking', 'economy', 'GDP']

In [33]:
df_csv_gdp = pd.read_csv(CSV_GDP_URL,  skiprows=range(0, 4))

In [34]:
df_csv_gdp.drop(columns=['Unnamed: 2', 'Unnamed: 5'], inplace = True)
df_csv_gdp.columns = col_names_gdp

In [35]:
df_csv_gdp.head()

,country_code,ranking,economy,GDP
0,USA,1,United States,"21,433,226"
1,CHN,2,China,"14,342,903"
2,JPN,3,Japan,"5,081,770"
3,DEU,4,Germany,"3,861,124"
4,IND,5,India,"2,868,929"


In [36]:
df_csv_gdp.columns

Index(['country_code', 'ranking', 'economy', 'GDP'], dtype='object')

### EDA